In [39]:
import pandas as pd
import numpy as np
from datetime import datetime as DateTime, timedelta as TimeDelta

In [40]:
docu = pd.ExcelFile('Covid Chile V2 PARA PRUEBAS.xlsx')

In [41]:
docuConfirma = pd.read_excel(docu,  sheet_name="COVID_CL_CONFIRMA")
docuMuertes = pd.read_excel(docu,  sheet_name="COVID_CL_MUERTE")
docuRecuperados = pd.read_excel(docu,  sheet_name="COVID_CL_RECUPERA")
#docuFinal = pd.read_excel(docu,  sheet_name="FINAL")

In [42]:
docuConfirma.dropna(axis = 0, how = 'all', inplace = True)
if(docuConfirma.columns[0] == 'Unnamed: 0'):
    docuConfirma.columns = docuConfirma.iloc[0]
    docuConfirma = docuConfirma.iloc[1:,].reindex()
    
docuMuertes.dropna(axis = 0, how = 'all', inplace = True)
if(docuMuertes.columns[0] == 'Unnamed: 0'):
    docuMuertes.columns = docuMuertes.iloc[0]
    docuMuertes = docuMuertes.iloc[1:,].reindex()
    
docuRecuperados.dropna(axis = 0, how = 'all', inplace = True)
if(docuRecuperados.columns[0] == 'Unnamed: 0'):
    docuRecuperados.columns = docuRecuperados.iloc[0]
    docuRecuperados = docuRecuperados.iloc[1:,].reindex()

In [43]:
#dfFecha = pd.DataFrame({"Fecha" : docuConfirma["Fecha"]})
#res = dfFecha.groupby(['Fecha']).sum().reset_index()
fechaMinima = docuConfirma.Fecha.min()
fechaMaxima = docuConfirma.Fecha.max() + TimeDelta(days=5)

listaFechas = []
listaFechas = pd.date_range(start=fechaMinima,end=fechaMaxima)

df3 = pd.DataFrame({"Region" : docuConfirma["Region"], "Comuna" : docuConfirma["Comuna"], "Codigo comuna" : docuConfirma["ID_Comuna"]})
res2 = df3.groupby(["Region","Comuna", "Codigo comuna"]).sum().reset_index()

In [44]:
lista = []
for Comuna in res2["Comuna"]:
    for Fecha in listaFechas:
        lista.append(Fecha)
        
res3 = pd.concat([res2]*len(listaFechas))
res3['Fecha'] = lista
res3['Casos Diarios'] = 0
res3['Fallecidos Diarios'] = 0
res3['Recuperados Diarios'] = 0

In [45]:
df4 = pd.DataFrame({"Region" : docuConfirma["Region"], "Comuna" : docuConfirma["Comuna"],"Codigo comuna" : docuConfirma["ID_Comuna"] ,"Fecha" : docuConfirma["Fecha"], "Casos Diarios" : 0})

resConfirma4 = df4.groupby(["Region","Comuna","Fecha",'Codigo comuna']).count().reset_index()
resConfirma4.insert(4, "Fallecidos Diarios", 0, True)
resConfirma4.insert(5, "Recuperados Diarios", 0, True)
resConfirma5 = pd.concat([ res3 ,  resConfirma4 ])
resConfirma6 = resConfirma5.groupby(['Region','Comuna','Fecha','Codigo comuna']).sum().reset_index()

In [46]:
df5 = pd.DataFrame({"Region" : docuMuertes["Region"], "Comuna" : docuMuertes["Comuna"], "Codigo comuna" : docuMuertes["ID_Comuna"] ,"Fecha" : docuMuertes["Fecha"], "Fallecidos Diarios" : 0})
resMuerte4 = df5.groupby(["Region","Comuna",'Codigo comuna',"Fecha"]).count().reset_index()
resMuerte4.insert(4, "Casos Diarios", 0, True)
resMuerte4.insert(6, "Recuperados Diarios", 0, True) 
resMuerte5 = pd.concat([ resConfirma6 ,  resMuerte4])
resMuerte6 = resMuerte5.groupby(['Region','Comuna','Fecha','Codigo comuna']).sum().reset_index()

In [47]:
df6 = pd.DataFrame({"Region" : docuRecuperados["Region"], "Comuna" : docuRecuperados["Comuna"], "Codigo comuna" : docuRecuperados["ID_Comuna"] ,"Fecha" : docuRecuperados["Fecha"], "Recuperados Diarios" : 0})
resRecupera4 = df6.groupby(["Region","Comuna",'Codigo comuna',"Fecha"]).count().reset_index()
resRecupera4.insert(4, "Casos Diarios", 0, True)
resRecupera4.insert(5, "Fallecidos Diarios", 0, True) 
resTotal = pd.concat([ resMuerte6 ,  resRecupera4])
resTotal = resTotal.groupby(['Region','Comuna','Fecha','Codigo comuna']).sum().reset_index()

In [48]:
#resTotal["Casos Acumulados"] = resTotal['Casos Diarios']
resTotal.insert(5, "Casos Acumulados", resTotal['Casos Diarios'], True)
resTotal.insert(6, "Muertes", resTotal['Fallecidos Diarios'], True)
resTotal.insert(7, "Recuperados", resTotal['Recuperados Diarios'], True) 
resTotal.insert(7, "Casos Activos", resTotal['Casos Acumulados'], True)

for i in range(1, len(resTotal)):
    if(resTotal.loc[i, 'Comuna'] == resTotal.loc[i-1, 'Comuna']):
        resTotal.loc[i, 'Casos Acumulados'] = resTotal.loc[i, 'Casos Acumulados'] + resTotal.loc[i-1, 'Casos Acumulados']
        resTotal.loc[i, 'Muertes'] = resTotal.loc[i, 'Muertes'] + resTotal.loc[i-1, 'Muertes']
        resTotal.loc[i, 'Recuperados'] = resTotal.loc[i, 'Recuperados'] + resTotal.loc[i-1, 'Recuperados']
        resTotal.loc[i, 'Casos Activos'] = (resTotal.loc[i, 'Casos Acumulados'] - resTotal.loc[i, 'Muertes']) - resTotal.loc[i, 'Recuperados']

In [49]:
resTotal

,Region,Comuna,Fecha,Codigo comuna,Casos Diarios,Casos Acumulados,Muertes,Casos Activos,Recuperados,Fallecidos Diarios,Recuperados Diarios
0,Antofagasta,Antofagasta,2020-03-03,2101,0,0,0,0,0,0,0
1,Antofagasta,Antofagasta,2020-03-04,2101,0,0,0,0,0,0,0
2,Antofagasta,Antofagasta,2020-03-05,2101,0,0,0,0,0,0,0
3,Antofagasta,Antofagasta,2020-03-06,2101,0,0,0,0,0,0,0
4,Antofagasta,Antofagasta,2020-03-07,2101,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
22091,Ñuble,Ñiquén,2020-05-10,16303,0,3,1,2,0,0,0
22092,Ñuble,Ñiquén,2020-05-11,16303,0,3,1,2,0,0,0
22093,Ñuble,Ñiquén,2020-05-12,16303,0,3,1,2,0,0,0
22094,Ñuble,Ñiquén,2020-05-13,16303,0,3,1,2,0,0,0


In [50]:
resTotal.to_csv("FINAL.csv", index=False)

In [197]:
resTotal[resTotal['Casos Diarios'] > 0].groupby(['Region', 'Comuna']).tail(1).index[0]

66